### Reprocessing Dataset

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN", "NAME"], inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df["APPLICATION_TYPE"].value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Visualize the value counts of APPLICATION_TYPE
application_type_counts.plot.density()

In [5]:
# Reducing number of bins used in application type
replace_application = list(application_type_counts[application_type_counts < 27037].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     7262
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2190        1
C1580        1
C1248        1
C4120        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Reducing number of bins used in classification
replace_class = list(classification_counts[classification_counts < 4837].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [8]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T3,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T3,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Optimizing the model

In [13]:
# Testing original model with new inputs
input_features = len(X_train[0])
layer1 = 8
layer2 = 5

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(
    tf.keras.layers.Dense(units=layer1, input_dim=input_features, activation="relu")
)

# Second hidden layer
nn1.add(
    tf.keras.layers.Dense(units=layer2, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 280       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 331
Trainable params: 331
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("analysis/checkpoints/",exist_ok=True)
checkpoint_path = "analysis/checkpoints/weights.{epoch:02d}.hdf5"

In [16]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Train the model
fit_model = nn1.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
  1/804 [..............................] - ETA: 3:28 - loss: 67479.6172 - accuracy: 0.6562
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
101/804 [==>...........................] - ETA: 0s - loss: 130123.2891 - accuracy: 0.5328 
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
201/804 [======>.......................] - ETA: 0s - loss: 74519.9531 - accuracy: 0.5330 
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
301/804 [==========>...................] - ETA: 0s - loss: 52713.6055 - accuracy: 0.5224
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
401/804 [=============>................] - ETA: 0s - loss: 40892.1016 - accuracy: 0.5100
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
501/804 [=================>............] - ETA: 0s - loss: 34138.6953 - accuracy: 0.5024
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
601/804 [=====================>........] - ETA: 0s - 

244/804 [========>.....................] - ETA: 0s - loss: 0.6910 - accuracy: 0.5333
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
322/804 [===========>..................] - ETA: 0s - loss: 0.6910 - accuracy: 0.5329
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
400/804 [=============>................] - ETA: 0s - loss: 0.6909 - accuracy: 0.5340
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
481/804 [================>.............] - ETA: 0s - loss: 0.6907 - accuracy: 0.5349
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
577/804 [====================>.........] - ETA: 0s - loss: 0.6910 - accuracy: 0.5328
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
677/804 [========================>.....] - ETA: 0s - loss: 0.6911 - accuracy: 0.5320
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
804/804 [==============================] - 0s 605us/step - loss: 0.6911 - accuracy: 0.5321
Epo

432/804 [===============>..............] - ETA: 0s - loss: 0.6912 - accuracy: 0.5312
Epoch 00013: saving model to analysis/checkpoints\weights.13.hdf5
519/804 [==================>...........] - ETA: 0s - loss: 0.6913 - accuracy: 0.5308
Epoch 00013: saving model to analysis/checkpoints\weights.13.hdf5
604/804 [=====================>........] - ETA: 0s - loss: 0.6915 - accuracy: 0.5290
Epoch 00013: saving model to analysis/checkpoints\weights.13.hdf5
683/804 [========================>.....] - ETA: 0s - loss: 0.6913 - accuracy: 0.5310
Epoch 00013: saving model to analysis/checkpoints\weights.13.hdf5
804/804 [==============================] - 0s 592us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 14/100
  1/804 [..............................] - ETA: 0s - loss: 0.6870 - accuracy: 0.5625
Epoch 00014: saving model to analysis/checkpoints\weights.14.hdf5
101/804 [==>...........................] - ETA: 0s - loss: 0.6909 - accuracy: 0.5340
Epoch 00014: saving model to analysis/checkpoints\weight

529/804 [==================>...........] - ETA: 0s - loss: 0.6917 - accuracy: 0.5277
Epoch 00019: saving model to analysis/checkpoints\weights.19.hdf5
719/804 [=========================>....] - ETA: 0s - loss: 0.6912 - accuracy: 0.5318
Epoch 00019: saving model to analysis/checkpoints\weights.19.hdf5
804/804 [==============================] - 1s 691us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 20/100
  1/804 [..............................] - ETA: 0s - loss: 0.6739 - accuracy: 0.6562
Epoch 00020: saving model to analysis/checkpoints\weights.20.hdf5
 72/804 [=>............................] - ETA: 0s - loss: 0.6897 - accuracy: 0.5421
Epoch 00020: saving model to analysis/checkpoints\weights.20.hdf5
140/804 [====>.........................] - ETA: 0s - loss: 0.6906 - accuracy: 0.5359
Epoch 00020: saving model to analysis/checkpoints\weights.20.hdf5
226/804 [=======>......................] - ETA: 0s - loss: 0.6906 - accuracy: 0.5358
Epoch 00020: saving model to analysis/checkpoints\weight

803/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5322
Epoch 00025: saving model to analysis/checkpoints\weights.25.hdf5
804/804 [==============================] - 0s 594us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 26/100
  1/804 [..............................] - ETA: 1s - loss: 0.6872 - accuracy: 0.5625
Epoch 00026: saving model to analysis/checkpoints\weights.26.hdf5
101/804 [==>...........................] - ETA: 0s - loss: 0.6901 - accuracy: 0.5396
Epoch 00026: saving model to analysis/checkpoints\weights.26.hdf5
201/804 [======>.......................] - ETA: 0s - loss: 0.6909 - accuracy: 0.5336
Epoch 00026: saving model to analysis/checkpoints\weights.26.hdf5
301/804 [==========>...................] - ETA: 0s - loss: 0.6911 - accuracy: 0.5326
Epoch 00026: saving model to analysis/checkpoints\weights.26.hdf5
401/804 [=============>................] - ETA: 0s - loss: 0.6910 - accuracy: 0.5333
Epoch 00026: saving model to analysis/checkpoints\weight

...

317/804 [==========>...................] - ETA: 0s - loss: 0.6911 - accuracy: 0.5326
Epoch 00075: saving model to analysis/checkpoints\weights.75.hdf5
504/804 [=================>............] - ETA: 0s - loss: 0.6910 - accuracy: 0.5328
Epoch 00075: saving model to analysis/checkpoints\weights.75.hdf5
579/804 [====================>.........] - ETA: 0s - loss: 0.6912 - accuracy: 0.5318
Epoch 00075: saving model to analysis/checkpoints\weights.75.hdf5
651/804 [=======================>......] - ETA: 0s - loss: 0.6911 - accuracy: 0.5325
Epoch 00075: saving model to analysis/checkpoints\weights.75.hdf5
718/804 [=========================>....] - ETA: 0s - loss: 0.6910 - accuracy: 0.5329
Epoch 00075: saving model to analysis/checkpoints\weights.75.hdf5
804/804 [==============================] - 1s 633us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 76/100
  1/804 [..............................] - ETA: 0s - loss: 0.6831 - accuracy: 0.5938
Epoch 00076: saving model to analysis/checkpoints\weight

528/804 [==================>...........] - ETA: 0s - loss: 0.6913 - accuracy: 0.5310
Epoch 00081: saving model to analysis/checkpoints\weights.81.hdf5
614/804 [=====================>........] - ETA: 0s - loss: 0.6912 - accuracy: 0.5315
Epoch 00081: saving model to analysis/checkpoints\weights.81.hdf5
687/804 [========================>.....] - ETA: 0s - loss: 0.6913 - accuracy: 0.5311
Epoch 00081: saving model to analysis/checkpoints\weights.81.hdf5
804/804 [==============================] - 0s 592us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 82/100
  1/804 [..............................] - ETA: 0s - loss: 0.7166 - accuracy: 0.3438
Epoch 00082: saving model to analysis/checkpoints\weights.82.hdf5
 91/804 [==>...........................] - ETA: 0s - loss: 0.6917 - accuracy: 0.5278
Epoch 00082: saving model to analysis/checkpoints\weights.82.hdf5
276/804 [=========>....................] - ETA: 0s - loss: 0.6918 - accuracy: 0.5267
Epoch 00082: saving model to analysis/checkpoints\weight

745/804 [==========================>...] - ETA: 0s - loss: 0.6911 - accuracy: 0.5320
Epoch 00087: saving model to analysis/checkpoints\weights.87.hdf5
804/804 [==============================] - 1s 642us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 88/100
  1/804 [..............................] - ETA: 0s - loss: 0.6993 - accuracy: 0.4688
Epoch 00088: saving model to analysis/checkpoints\weights.88.hdf5
 94/804 [==>...........................] - ETA: 0s - loss: 0.6912 - accuracy: 0.5316
Epoch 00088: saving model to analysis/checkpoints\weights.88.hdf5
181/804 [=====>........................] - ETA: 0s - loss: 0.6908 - accuracy: 0.5344
Epoch 00088: saving model to analysis/checkpoints\weights.88.hdf5
268/804 [=========>....................] - ETA: 0s - loss: 0.6915 - accuracy: 0.5289
Epoch 00088: saving model to analysis/checkpoints\weights.88.hdf5
357/804 [============>.................] - ETA: 0s - loss: 0.6918 - accuracy: 0.5269
Epoch 00088: saving model to analysis/checkpoints\weight

  1/804 [..............................] - ETA: 0s - loss: 0.6873 - accuracy: 0.5625
Epoch 00094: saving model to analysis/checkpoints\weights.94.hdf5
 99/804 [==>...........................] - ETA: 0s - loss: 0.6928 - accuracy: 0.5180
Epoch 00094: saving model to analysis/checkpoints\weights.94.hdf5
187/804 [=====>........................] - ETA: 0s - loss: 0.6921 - accuracy: 0.5237
Epoch 00094: saving model to analysis/checkpoints\weights.94.hdf5
276/804 [=========>....................] - ETA: 0s - loss: 0.6919 - accuracy: 0.5256
Epoch 00094: saving model to analysis/checkpoints\weights.94.hdf5
364/804 [============>.................] - ETA: 0s - loss: 0.6916 - accuracy: 0.5282
Epoch 00094: saving model to analysis/checkpoints\weights.94.hdf5
449/804 [===============>..............] - ETA: 0s - loss: 0.6915 - accuracy: 0.5289
Epoch 00094: saving model to analysis/checkpoints\weights.94.hdf5
537/804 [===================>..........] - ETA: 0s - loss: 0.6913 - accuracy: 0.5303
Epoch 000

187/804 [=====>........................] - ETA: 0s - loss: 0.6901 - accuracy: 0.5399
Epoch 00100: saving model to analysis/checkpoints\weights.100.hdf5
273/804 [=========>....................] - ETA: 0s - loss: 0.6909 - accuracy: 0.5338
Epoch 00100: saving model to analysis/checkpoints\weights.100.hdf5
361/804 [============>.................] - ETA: 0s - loss: 0.6911 - accuracy: 0.5319
Epoch 00100: saving model to analysis/checkpoints\weights.100.hdf5
448/804 [===============>..............] - ETA: 0s - loss: 0.6913 - accuracy: 0.5303
Epoch 00100: saving model to analysis/checkpoints\weights.100.hdf5
536/804 [===================>..........] - ETA: 0s - loss: 0.6911 - accuracy: 0.5318
Epoch 00100: saving model to analysis/checkpoints\weights.100.hdf5
625/804 [======================>.......] - ETA: 0s - loss: 0.6910 - accuracy: 0.5328
Epoch 00100: saving model to analysis/checkpoints\weights.100.hdf5
713/804 [=========================>....] - ETA: 0s - loss: 0.6913 - accuracy: 0.5308
Epo

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6892 - accuracy: 0.5332
Loss: 0.6892154812812805, Accuracy: 0.5331778526306152


In [24]:
nn1.save("AlphabetSoupCharity_Optimization1.h5")

In [20]:
# Adding additional neurons to model
input_features = len(X_train[0])
layer1 = 24
layer2 = 16

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=layer1, input_dim=input_features, activation="relu")
)

# Second hidden layer
nn2.add(
    tf.keras.layers.Dense(units=layer2, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 24)                840       
_________________________________________________________________
dense_4 (Dense)              (None, 16)                400       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 1,257
Trainable params: 1,257
Non-trainable params: 0
_________________________________________________________________


In [21]:
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Train the model
fit_model = nn2.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
 99/804 [==>...........................] - ETA: 0s - loss: 8506.0947 - accuracy: 0.5044  
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
184/804 [=====>........................] - ETA: 0s - loss: 6826.4502 - accuracy: 0.5105
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
261/804 [========>.....................] - ETA: 0s - loss: 99483.0312 - accuracy: 0.5122
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
347/804 [===========>..................] - ETA: 0s - loss: 96908.9766 - accuracy: 0.5114
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
429/804 [===============>..............] - ETA: 0s - loss: 79611.1328 - accuracy: 0.5066
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
513/804 [==================>...........] - ETA: 0s - loss: 66822.3828 - accuracy: 0.5068
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
601/804 [=====================>........] - ETA: 0s - loss:

 77/804 [=>............................] - ETA: 0s - loss: 756.6879 - accuracy: 0.5061
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
177/804 [=====>........................] - ETA: 0s - loss: 602.7421 - accuracy: 0.5252
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
277/804 [=========>....................] - ETA: 0s - loss: 7208.4185 - accuracy: 0.5215
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
468/804 [================>.............] - ETA: 0s - loss: 4625.4702 - accuracy: 0.5269
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
556/804 [===================>..........] - ETA: 0s - loss: 4051.3826 - accuracy: 0.5317
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
601/804 [=====================>........] - ETA: 0s - loss: 4956.9258 - accuracy: 0.5309
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
769/804 [===========================>..] - ETA: 0s - loss: 28392.2832 - accura

163/804 [=====>........................] - ETA: 0s - loss: 44811.0547 - accuracy: 0.5424
Epoch 00013: saving model to analysis/checkpoints\weights.13.hdf5
253/804 [========>.....................] - ETA: 0s - loss: 33167.1562 - accuracy: 0.5372
Epoch 00013: saving model to analysis/checkpoints\weights.13.hdf5
353/804 [============>.................] - ETA: 0s - loss: 25929.9199 - accuracy: 0.5364
Epoch 00013: saving model to analysis/checkpoints\weights.13.hdf5
453/804 [===============>..............] - ETA: 0s - loss: 20417.9199 - accuracy: 0.5288
Epoch 00013: saving model to analysis/checkpoints\weights.13.hdf5
553/804 [===================>..........] - ETA: 0s - loss: 17208.6953 - accuracy: 0.5242
Epoch 00013: saving model to analysis/checkpoints\weights.13.hdf5
653/804 [=======================>......] - ETA: 0s - loss: 20919.8066 - accuracy: 0.5176
Epoch 00013: saving model to analysis/checkpoints\weights.13.hdf5
804/804 [==============================] - 0s 590us/step - loss: 18538

240/804 [=======>......................] - ETA: 0s - loss: 20793.5547 - accuracy: 0.5236
Epoch 00019: saving model to analysis/checkpoints\weights.19.hdf5
329/804 [===========>..................] - ETA: 0s - loss: 15247.8574 - accuracy: 0.5289
Epoch 00019: saving model to analysis/checkpoints\weights.19.hdf5
429/804 [===============>..............] - ETA: 0s - loss: 11781.7119 - accuracy: 0.5355
Epoch 00019: saving model to analysis/checkpoints\weights.19.hdf5
626/804 [======================>.......] - ETA: 0s - loss: 33883.7109 - accuracy: 0.5251
Epoch 00019: saving model to analysis/checkpoints\weights.19.hdf5
708/804 [=========================>....] - ETA: 0s - loss: 30085.9258 - accuracy: 0.5284
Epoch 00019: saving model to analysis/checkpoints\weights.19.hdf5
804/804 [==============================] - 0s 608us/step - loss: 26510.9434 - accuracy: 0.5315
Epoch 20/100
  1/804 [..............................] - ETA: 0s - loss: 19.6240 - accuracy: 0.7500
Epoch 00020: saving model to an

317/804 [==========>...................] - ETA: 0s - loss: 21008.0449 - accuracy: 0.5099
Epoch 00025: saving model to analysis/checkpoints\weights.25.hdf5
405/804 [==============>...............] - ETA: 0s - loss: 16714.8262 - accuracy: 0.5288
Epoch 00025: saving model to analysis/checkpoints\weights.25.hdf5
597/804 [=====================>........] - ETA: 0s - loss: 11594.4160 - accuracy: 0.5422
Epoch 00025: saving model to analysis/checkpoints\weights.25.hdf5
673/804 [========================>.....] - ETA: 0s - loss: 10465.6191 - accuracy: 0.5411
Epoch 00025: saving model to analysis/checkpoints\weights.25.hdf5
755/804 [===========================>..] - ETA: 0s - loss: 9413.3896 - accuracy: 0.5489 
Epoch 00025: saving model to analysis/checkpoints\weights.25.hdf5
804/804 [==============================] - 1s 629us/step - loss: 8852.6309 - accuracy: 0.5533
Epoch 26/100
  1/804 [..............................] - ETA: 1s - loss: 14.1602 - accuracy: 0.6875
Epoch 00026: saving model to ana

...

609/804 [=====================>........] - ETA: 0s - loss: 0.6909 - accuracy: 0.5340
Epoch 00074: saving model to analysis/checkpoints\weights.74.hdf5
804/804 [==============================] - 0s 584us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 75/100
  1/804 [..............................] - ETA: 0s - loss: 0.6912 - accuracy: 0.5312
Epoch 00075: saving model to analysis/checkpoints\weights.75.hdf5
 77/804 [=>............................] - ETA: 0s - loss: 0.6920 - accuracy: 0.5252
Epoch 00075: saving model to analysis/checkpoints\weights.75.hdf5
159/804 [====>.........................] - ETA: 0s - loss: 0.6910 - accuracy: 0.5332
Epoch 00075: saving model to analysis/checkpoints\weights.75.hdf5
243/804 [========>.....................] - ETA: 0s - loss: 0.6907 - accuracy: 0.5358
Epoch 00075: saving model to analysis/checkpoints\weights.75.hdf5
326/804 [===========>..................] - ETA: 0s - loss: 0.6908 - accuracy: 0.5351
Epoch 00075: saving model to analysis/checkpoints\weight

804/804 [==============================] - 0s 584us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 81/100
  1/804 [..............................] - ETA: 0s - loss: 0.6912 - accuracy: 0.5312
Epoch 00081: saving model to analysis/checkpoints\weights.81.hdf5
 81/804 [==>...........................] - ETA: 0s - loss: 0.6922 - accuracy: 0.5231
Epoch 00081: saving model to analysis/checkpoints\weights.81.hdf5
181/804 [=====>........................] - ETA: 0s - loss: 0.6921 - accuracy: 0.5238
Epoch 00081: saving model to analysis/checkpoints\weights.81.hdf5
281/804 [=========>....................] - ETA: 0s - loss: 0.6917 - accuracy: 0.5281
Epoch 00081: saving model to analysis/checkpoints\weights.81.hdf5
381/804 [=============>................] - ETA: 0s - loss: 0.6917 - accuracy: 0.5276
Epoch 00081: saving model to analysis/checkpoints\weights.81.hdf5
481/804 [================>.............] - ETA: 0s - loss: 0.6914 - accuracy: 0.5305
Epoch 00081: saving model to analysis/checkpoints\weight

 79/804 [=>............................] - ETA: 0s - loss: 0.6905 - accuracy: 0.5368
Epoch 00087: saving model to analysis/checkpoints\weights.87.hdf5
164/804 [=====>........................] - ETA: 0s - loss: 0.6893 - accuracy: 0.5448
Epoch 00087: saving model to analysis/checkpoints\weights.87.hdf5
257/804 [========>.....................] - ETA: 0s - loss: 0.6896 - accuracy: 0.5427
Epoch 00087: saving model to analysis/checkpoints\weights.87.hdf5
357/804 [============>.................] - ETA: 0s - loss: 0.6904 - accuracy: 0.5376
Epoch 00087: saving model to analysis/checkpoints\weights.87.hdf5
457/804 [================>.............] - ETA: 0s - loss: 0.6909 - accuracy: 0.5336
Epoch 00087: saving model to analysis/checkpoints\weights.87.hdf5
557/804 [===================>..........] - ETA: 0s - loss: 0.6909 - accuracy: 0.5341
Epoch 00087: saving model to analysis/checkpoints\weights.87.hdf5
657/804 [=======================>......] - ETA: 0s - loss: 0.6910 - accuracy: 0.5331
Epoch 000

245/804 [========>.....................] - ETA: 0s - loss: 0.6903 - accuracy: 0.5381
Epoch 00093: saving model to analysis/checkpoints\weights.93.hdf5
333/804 [===========>..................] - ETA: 0s - loss: 0.6906 - accuracy: 0.5366
Epoch 00093: saving model to analysis/checkpoints\weights.93.hdf5
433/804 [===============>..............] - ETA: 0s - loss: 0.6907 - accuracy: 0.5354
Epoch 00093: saving model to analysis/checkpoints\weights.93.hdf5
533/804 [==================>...........] - ETA: 0s - loss: 0.6911 - accuracy: 0.5325
Epoch 00093: saving model to analysis/checkpoints\weights.93.hdf5
633/804 [======================>.......] - ETA: 0s - loss: 0.6911 - accuracy: 0.5328
Epoch 00093: saving model to analysis/checkpoints\weights.93.hdf5
804/804 [==============================] - 0s 590us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 94/100
  1/804 [..............................] - ETA: 0s - loss: 0.6691 - accuracy: 0.7188
Epoch 00094: saving model to analysis/checkpoints\weight

409/804 [==============>...............] - ETA: 0s - loss: 0.6909 - accuracy: 0.5340
Epoch 00099: saving model to analysis/checkpoints\weights.99.hdf5
509/804 [=================>............] - ETA: 0s - loss: 0.6909 - accuracy: 0.5339
Epoch 00099: saving model to analysis/checkpoints\weights.99.hdf5
609/804 [=====================>........] - ETA: 0s - loss: 0.6911 - accuracy: 0.5325
Epoch 00099: saving model to analysis/checkpoints\weights.99.hdf5
804/804 [==============================] - 0s 593us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 100/100
  1/804 [..............................] - ETA: 0s - loss: 0.7062 - accuracy: 0.4062
Epoch 00100: saving model to analysis/checkpoints\weights.100.hdf5
 79/804 [=>............................] - ETA: 0s - loss: 0.6911 - accuracy: 0.5324
Epoch 00100: saving model to analysis/checkpoints\weights.100.hdf5
158/804 [====>.........................] - ETA: 0s - loss: 0.6917 - accuracy: 0.5273
Epoch 00100: saving model to analysis/checkpoints\wei

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.8543 - accuracy: 0.6645
Loss: 0.8542798161506653, Accuracy: 0.6644898056983948


In [25]:
nn2.save("AlphabetSoupCharity_Optimization2.h5")

In [31]:
# Adding an additional hidden layer to model
input_features = len(X_train[0])
layer1 = 12
layer2 = 8
layer3 = 6
nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=layer1, input_dim=input_features, activation="relu")
)

# Second hidden layer
nn3.add(
    tf.keras.layers.Dense(units=layer2, activation="relu"))

nn3.add(
    tf.keras.layers.Dense(units=layer3, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 12)                420       
_________________________________________________________________
dense_11 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_12 (Dense)             (None, 6)                 54        
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 7         
Total params: 585
Trainable params: 585
Non-trainable params: 0
_________________________________________________________________


In [32]:
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [33]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Train the model
fit_model = nn3.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
  1/804 [..............................] - ETA: 3:15 - loss: 7627.3594 - accuracy: 0.5000
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
101/804 [==>...........................] - ETA: 0s - loss: 18495.2207 - accuracy: 0.4734 
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
201/804 [======>.......................] - ETA: 0s - loss: 12306.9922 - accuracy: 0.4824
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
301/804 [==========>...................] - ETA: 0s - loss: 8719.0273 - accuracy: 0.4770 
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
401/804 [=============>................] - ETA: 0s - loss: 16992.3809 - accuracy: 0.4902
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
501/804 [=================>............] - ETA: 0s - loss: 16608.4473 - accuracy: 0.4951
Epoch 00001: saving model to analysis/checkpoints\weights.01.hdf5
601/804 [=====================>........] - ETA: 0s - los

 77/804 [=>............................] - ETA: 0s - loss: 321.1307 - accuracy: 0.4647
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
276/804 [=========>....................] - ETA: 0s - loss: 90.1035 - accuracy: 0.5172 
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
350/804 [============>.................] - ETA: 0s - loss: 71.2023 - accuracy: 0.5210
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
414/804 [==============>...............] - ETA: 0s - loss: 60.3042 - accuracy: 0.5226
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
570/804 [====================>.........] - ETA: 0s - loss: 43.9917 - accuracy: 0.5255
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
648/804 [=======================>......] - ETA: 0s - loss: 38.7808 - accuracy: 0.5255
Epoch 00007: saving model to analysis/checkpoints\weights.07.hdf5
729/804 [==========================>...] - ETA: 0s - loss: 34.5495 - accuracy: 0.5254


253/804 [========>.....................] - ETA: 0s - loss: 0.6910 - accuracy: 0.5333
Epoch 00013: saving model to analysis/checkpoints\weights.13.hdf5
353/804 [============>.................] - ETA: 0s - loss: 0.6912 - accuracy: 0.5317
Epoch 00013: saving model to analysis/checkpoints\weights.13.hdf5
546/804 [===================>..........] - ETA: 0s - loss: 0.6909 - accuracy: 0.5338
Epoch 00013: saving model to analysis/checkpoints\weights.13.hdf5
608/804 [=====================>........] - ETA: 0s - loss: 0.6911 - accuracy: 0.5328
Epoch 00013: saving model to analysis/checkpoints\weights.13.hdf5
673/804 [========================>.....] - ETA: 0s - loss: 0.6911 - accuracy: 0.5327
Epoch 00013: saving model to analysis/checkpoints\weights.13.hdf5
804/804 [==============================] - 1s 661us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 14/100
 36/804 [>.............................] - ETA: 1s - loss: 0.6926 - accuracy: 0.5200
Epoch 00014: saving model to analysis/checkpoints\weight

449/804 [===============>..............] - ETA: 0s - loss: 0.6913 - accuracy: 0.5307
Epoch 00019: saving model to analysis/checkpoints\weights.19.hdf5
530/804 [==================>...........] - ETA: 0s - loss: 0.6914 - accuracy: 0.5301
Epoch 00019: saving model to analysis/checkpoints\weights.19.hdf5
629/804 [======================>.......] - ETA: 0s - loss: 0.6914 - accuracy: 0.5299
Epoch 00019: saving model to analysis/checkpoints\weights.19.hdf5
804/804 [==============================] - 1s 638us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 20/100
  1/804 [..............................] - ETA: 0s - loss: 0.6640 - accuracy: 0.7188
Epoch 00020: saving model to analysis/checkpoints\weights.20.hdf5
 79/804 [=>............................] - ETA: 0s - loss: 0.6899 - accuracy: 0.5407
Epoch 00020: saving model to analysis/checkpoints\weights.20.hdf5
148/804 [====>.........................] - ETA: 0s - loss: 0.6913 - accuracy: 0.5312
Epoch 00020: saving model to analysis/checkpoints\weight

680/804 [========================>.....] - ETA: 0s - loss: 0.6914 - accuracy: 0.5302
Epoch 00025: saving model to analysis/checkpoints\weights.25.hdf5
752/804 [===========================>..] - ETA: 0s - loss: 0.6912 - accuracy: 0.5320
Epoch 00025: saving model to analysis/checkpoints\weights.25.hdf5
804/804 [==============================] - 1s 784us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 26/100
  1/804 [..............................] - ETA: 0s - loss: 0.7134 - accuracy: 0.3750
Epoch 00026: saving model to analysis/checkpoints\weights.26.hdf5
101/804 [==>...........................] - ETA: 0s - loss: 0.6913 - accuracy: 0.5312
Epoch 00026: saving model to analysis/checkpoints\weights.26.hdf5
201/804 [======>.......................] - ETA: 0s - loss: 0.6906 - accuracy: 0.5362
Epoch 00026: saving model to analysis/checkpoints\weights.26.hdf5
301/804 [==========>...................] - ETA: 0s - loss: 0.6906 - accuracy: 0.5361
Epoch 00026: saving model to analysis/checkpoints\weight

...

318/804 [==========>...................] - ETA: 0s - loss: 0.6913 - accuracy: 0.5312
Epoch 00075: saving model to analysis/checkpoints\weights.75.hdf5
405/804 [==============>...............] - ETA: 0s - loss: 0.6909 - accuracy: 0.5340
Epoch 00075: saving model to analysis/checkpoints\weights.75.hdf5
505/804 [=================>............] - ETA: 0s - loss: 0.6908 - accuracy: 0.5345
Epoch 00075: saving model to analysis/checkpoints\weights.75.hdf5
605/804 [=====================>........] - ETA: 0s - loss: 0.6910 - accuracy: 0.5334
Epoch 00075: saving model to analysis/checkpoints\weights.75.hdf5
705/804 [=========================>....] - ETA: 0s - loss: 0.6912 - accuracy: 0.5320
Epoch 00075: saving model to analysis/checkpoints\weights.75.hdf5
804/804 [==============================] - 0s 611us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 76/100
  1/804 [..............................] - ETA: 0s - loss: 0.6758 - accuracy: 0.6562
Epoch 00076: saving model to analysis/checkpoints\weight

481/804 [================>.............] - ETA: 0s - loss: 0.6918 - accuracy: 0.5270
Epoch 00081: saving model to analysis/checkpoints\weights.81.hdf5
581/804 [====================>.........] - ETA: 0s - loss: 0.6914 - accuracy: 0.5304
Epoch 00081: saving model to analysis/checkpoints\weights.81.hdf5
681/804 [========================>.....] - ETA: 0s - loss: 0.6915 - accuracy: 0.5296
Epoch 00081: saving model to analysis/checkpoints\weights.81.hdf5
804/804 [==============================] - 0s 596us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 82/100
  1/804 [..............................] - ETA: 0s - loss: 0.6913 - accuracy: 0.5312
Epoch 00082: saving model to analysis/checkpoints\weights.82.hdf5
 82/804 [==>...........................] - ETA: 0s - loss: 0.6926 - accuracy: 0.5225
Epoch 00082: saving model to analysis/checkpoints\weights.82.hdf5
177/804 [=====>........................] - ETA: 0s - loss: 0.6908 - accuracy: 0.5350
Epoch 00082: saving model to analysis/checkpoints\weight

712/804 [=========================>....] - ETA: 0s - loss: 0.6913 - accuracy: 0.5307
Epoch 00087: saving model to analysis/checkpoints\weights.87.hdf5
804/804 [==============================] - 0s 605us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 88/100
  1/804 [..............................] - ETA: 0s - loss: 0.6869 - accuracy: 0.5625
Epoch 00088: saving model to analysis/checkpoints\weights.88.hdf5
 81/804 [==>...........................] - ETA: 0s - loss: 0.6922 - accuracy: 0.5239
Epoch 00088: saving model to analysis/checkpoints\weights.88.hdf5
162/804 [=====>........................] - ETA: 0s - loss: 0.6910 - accuracy: 0.5334
Epoch 00088: saving model to analysis/checkpoints\weights.88.hdf5
253/804 [========>.....................] - ETA: 0s - loss: 0.6910 - accuracy: 0.5331
Epoch 00088: saving model to analysis/checkpoints\weights.88.hdf5
353/804 [============>.................] - ETA: 0s - loss: 0.6911 - accuracy: 0.5325
Epoch 00088: saving model to analysis/checkpoints\weight

  1/804 [..............................] - ETA: 0s - loss: 0.6829 - accuracy: 0.5938
Epoch 00094: saving model to analysis/checkpoints\weights.94.hdf5
 80/804 [=>............................] - ETA: 0s - loss: 0.6925 - accuracy: 0.5219
Epoch 00094: saving model to analysis/checkpoints\weights.94.hdf5
160/804 [====>.........................] - ETA: 0s - loss: 0.6919 - accuracy: 0.5270
Epoch 00094: saving model to analysis/checkpoints\weights.94.hdf5
243/804 [========>.....................] - ETA: 0s - loss: 0.6913 - accuracy: 0.5316
Epoch 00094: saving model to analysis/checkpoints\weights.94.hdf5
419/804 [==============>...............] - ETA: 0s - loss: 0.6913 - accuracy: 0.5312
Epoch 00094: saving model to analysis/checkpoints\weights.94.hdf5
498/804 [=================>............] - ETA: 0s - loss: 0.6914 - accuracy: 0.5302
Epoch 00094: saving model to analysis/checkpoints\weights.94.hdf5
579/804 [====================>.........] - ETA: 0s - loss: 0.6915 - accuracy: 0.5296
Epoch 000

163/804 [=====>........................] - ETA: 0s - loss: 0.6903 - accuracy: 0.5383
Epoch 00100: saving model to analysis/checkpoints\weights.100.hdf5
243/804 [========>.....................] - ETA: 0s - loss: 0.6906 - accuracy: 0.5365
Epoch 00100: saving model to analysis/checkpoints\weights.100.hdf5
324/804 [===========>..................] - ETA: 0s - loss: 0.6908 - accuracy: 0.5345
Epoch 00100: saving model to analysis/checkpoints\weights.100.hdf5
407/804 [==============>...............] - ETA: 0s - loss: 0.6909 - accuracy: 0.5338
Epoch 00100: saving model to analysis/checkpoints\weights.100.hdf5
505/804 [=================>............] - ETA: 0s - loss: 0.6909 - accuracy: 0.5338
Epoch 00100: saving model to analysis/checkpoints\weights.100.hdf5
605/804 [=====================>........] - ETA: 0s - loss: 0.6908 - accuracy: 0.5347
Epoch 00100: saving model to analysis/checkpoints\weights.100.hdf5
705/804 [=========================>....] - ETA: 0s - loss: 0.6911 - accuracy: 0.5327
Epo

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7364 - accuracy: 0.4713
Loss: 0.7363542318344116, Accuracy: 0.4712536334991455


In [35]:
nn3.save("AlphabetSoupCharity_Optimization3.h5")